In [1]:
#Installations 

In [2]:
%pip -q install langchain openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip show langchain

Name: langchain
Version: 0.0.228
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: c:\users\venka\appdata\local\programs\python\python310\lib\site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: gpt-index, llama-index
Note: you may need to restart the kernel to use updated packages.


In [5]:
#Imports
import os
import sys
import pandas as pd
import sqlite3

from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [6]:
# Set the OpenAI API key in the environment variable
os.environ["OPENAI_API_KEY"] = "*********************************"
df = pd.read_csv('knowledge/data.csv')


#Converting into CSV --> Database
conn = sqlite3.connect('database.db')
df.to_sql('table_name', conn, if_exists='replace')


1746661

In [7]:
def answerMe(q):
    # Set up the agent using the create_csv_agent function
    agent = create_csv_agent(OpenAI(temperature=0), 'knowledge/data.csv', verbose=True)

    #######
    agent.agent.llm_chain.prompt.template

    #Question
    return(agent.run(q))

In [8]:
def updateData():
    # Replace 'your_table_name' with the name of the table containing the data you want to retrieve
    # If you want to select specific columns, replace '*' with the column names separated by commas
    query = "SELECT * FROM table_name;"
    
    # Use pandas to read the data into a DataFrame
    df = pd.read_sql_query(query, conn)
    # Specify the path where you want to save the .csv file
    file_path = 'knowledge/data.csv'
    
    # Save the DataFrame to the .csv file
    df.to_csv(file_path, index=True)  # Set index=False to avoid writing the index column to the file

In [9]:
def sql_query(q):
    cursor = conn.cursor()
    cursor.execute(q)
    conn.commit()

    if q.strip().upper().startswith('SELECT'):
        # For SELECT queries, return the results
        results = cursor.fetchall()
        # updateData()
        return results
    elif q.strip().upper().startswith('INSERT'):
        # For INSERT queries, return a success message
        updateData()
        return "Data inserted successfully!"
    else:
        # For other queries (UPDATE, DELETE, etc.), return None
        updateData()
        return "ELSE Block executed"


In [10]:
##INPUT:
i=1
while(i):
    print("""
            0 - STOP
            1 - Normal Query
            2 - SQL Query""")
    i = int(input())
    if(i==0):
        break
    elif(i==1):
        tex = input("Enter the Question: ")
        print(answerMe(tex))
    elif(i==2):
        tex = input("Enter the Question: ")
        print(sql_query(tex))
    else:
        print("invalid input, TRY AGAIN!")


            0 - STOP
            1 - Normal Query
            2 - SQL Query


In [25]:
answerMe("what is the pollution in delhi?")



> Entering new  chain...
Thought: I need to find the pollution data for Delhi
Action: python_repl_ast
Action Input: df[df['City'] == 'Delhi']
Observation: Empty DataFrame
Columns: [Unnamed: 0, State Code, County Code, Site Num, Address, State, County, City, Date Local, NO2 Units, NO2 Mean, NO2 1st Max Value, NO2 1st Max Hour, NO2 AQI, O3 Units, O3 Mean, O3 1st Max Value, O3 1st Max Hour, O3 AQI, SO2 Units, SO2 Mean, SO2 1st Max Value, SO2 1st Max Hour, SO2 AQI, CO Units, CO Mean, CO 1st Max Value, CO 1st Max Hour, CO AQI]
Index: []

[0 rows x 29 columns]
Thought: The dataframe is empty, so there is no pollution data for Delhi
Final Answer: There is no pollution data for Delhi.

> Finished chain.


'There is no pollution data for Delhi.'

In [12]:
# print(sql_query('INSERT INTO table_name ("Phone Name", "Rating ?/5", "Number of Ratings", "RAM", "ROM/Storage", "Back/Rare Camera", "Front Camera", "Battery", "Processor", "Price in INR", "Date of Scraping") VALUES("Project ARA", 4.9, 95,701, "4 GB RAM", "64 GB ROM", "16MP + 5MP + 2MP + Low Light Sensor", "32MP Front Camera", "4000 mAh", "Helio P22 (MTK6762) Processor", "â‚¹10,999", "17-06-23")'))
# print(sql_query('INSERT INTO table_name ("Phone Name", "Rating ?/5") VALUES("Project ARA", 4.9)'))
